In [1]:
import sys
import os



import numpy as np
from SynthTempNetwork import Individual, SynthTempNetwork
from TemporalNetwork import ContTempNetwork, StaticTempNetwork
from FlowStability import FlowIntegralClustering
import pickle

import matplotlib.pyplot as plt
import matplotlib

from scipy.sparse import (lil_matrix, dok_matrix, diags, eye, isspmatrix_csr, isspmatrix,
                          csr_matrix, coo_matrix, csc_matrix)
from scipy.sparse.linalg import eigsh

Could not load sparse_dot_mkl. Will use scipy.sparse for matrix products.


In [2]:
lamdas=np.logspace(-4,2,20)
print(lamdas)

[1.00000000e-04 2.06913808e-04 4.28133240e-04 8.85866790e-04
 1.83298071e-03 3.79269019e-03 7.84759970e-03 1.62377674e-02
 3.35981829e-02 6.95192796e-02 1.43844989e-01 2.97635144e-01
 6.15848211e-01 1.27427499e+00 2.63665090e+00 5.45559478e+00
 1.12883789e+01 2.33572147e+01 4.83293024e+01 1.00000000e+02]


In [3]:
net = ContTempNetwork.load('fig3_network_short')

In [21]:
list_forw_flows = []

for i in lamdas:
    list_forw_flows.append(FlowIntegralClustering(T_inter_list=net.inter_T[i],
                                  time_list=net.times,
                                  verbose=True))

PID  14257  : computing T_list
PID  14257  : computing integral
PID  14257  :  0  over  1039  took 0.00s
PID  14257  :  1000  over  1039  took 0.32s
integral took 0.33s
PID  14257  : computing T_list
PID  14257  : computing integral
PID  14257  :  0  over  1039  took 0.00s
PID  14257  :  1000  over  1039  took 0.30s
integral took 0.32s
PID  14257  : computing T_list
PID  14257  : computing integral
PID  14257  :  0  over  1039  took 0.00s
PID  14257  :  1000  over  1039  took 0.31s
integral took 0.32s
PID  14257  : computing T_list
PID  14257  : computing integral
PID  14257  :  0  over  1039  took 0.00s
PID  14257  :  1000  over  1039  took 0.31s
integral took 0.32s
PID  14257  : computing T_list
PID  14257  : computing integral
PID  14257  :  0  over  1039  took 0.00s
PID  14257  :  1000  over  1039  took 0.31s
integral took 0.32s
PID  14257  : computing T_list
PID  14257  : computing integral
PID  14257  :  0  over  1039  took 0.00s
PID  14257  :  1000  over  1039  took 0.31s
integr

In [22]:
for i in range(len(lamdas)):
    list_forw_flows[i].find_louvain_clustering()

### Generating/Saving/Loading static networks

In [23]:
#Generating Networks

static_net_adj = net.compute_static_adjacency_matrix()

#weighted
#Combinatorial Laplacian
static_net_weighted_heat = StaticTempNetwork(times = net.times, adjacency = static_net_adj)
static_net_weighted_heat.compute_laplacian_matrices(random_walk = False)
for i in lamdas:
    static_net_weighted_heat.compute_inter_transition_matrices(lamda=i, dense_expm=False, use_sparse_stoch=False)
    static_net_weighted_heat.compute_transition_matrices(lamda=i)

#Random Walk Laplacian
static_net_weighted = StaticTempNetwork(times = net.times, adjacency = static_net_adj)
static_net_weighted.compute_laplacian_matrices(random_walk = True)
for i in lamdas:
    static_net_weighted.compute_inter_transition_matrices(lamda=i, dense_expm=False, use_sparse_stoch=False)
    static_net_weighted.compute_transition_matrices(lamda=i)


#unweighted
#entries are sum of durations of events, need to set to 1
static_net_adj = static_net_adj.toarray()
static_net_adj[static_net_adj != 0] = 1
static_net_adj = csr_matrix(static_net_adj)

#Combinatorial Laplacian
static_net_heat = StaticTempNetwork(times = net.times, adjacency = static_net_adj)
static_net_heat.compute_laplacian_matrices(random_walk = False)
for i in lamdas:
    static_net_heat.compute_inter_transition_matrices(lamda=i, dense_expm=False, use_sparse_stoch=False)
    static_net_heat.compute_transition_matrices(lamda=i)

#Random Walk Laplacian
static_net = StaticTempNetwork(times = net.times, adjacency = static_net_adj)
static_net.compute_laplacian_matrices(random_walk = True)
for i in lamdas:
    static_net.compute_inter_transition_matrices(lamda=i, dense_expm=False, use_sparse_stoch=False)
    static_net.compute_transition_matrices(lamda=i)

/Users/samuelkoovely/Documents/GitHub/flow_stability/flow_stability_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:395: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/Users/samuelkoovely/Documents/GitHub/flow_stability/flow_stability_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:285: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


In [24]:
#Saving Networks
static_net_weighted_heat.save('fig3_network_static_weighted_heat')
static_net_weighted.save('fig3_network_static_weighted')
static_net_heat.save('fig3_network_static_heat')
static_net.save('fig3_network_static')

In [25]:
#Loading Networks
static_net_weighted_heat = StaticTempNetwork.load('fig3_network_static_weighted_heat')
static_net_weighted = StaticTempNetwork.load('fig3_network_static_weighted')
static_net_heat = StaticTempNetwork.load('fig3_network_static_heat')
static_net = StaticTempNetwork.load('fig3_network_static')

In [26]:
def one_time(s):
    one_time_s = []
    for key in s.keys():
        for i in s[key]:
            one_time_s.append(i)
    return one_time_s

# Static Network, Heat Diffusion

### Entropy Rate

In [ ]:
for i in lamdas:
    static_net_heat.compute_entropy(lamda=i, force_csr=True, time_domain = static_net_heat.times[-1:])

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(lamdas, one_time(static_net_heat.S))

plt.subplot(1, 2, 2)
plt.plot(lamdas, one_time(static_net_heat.S))
plt.yscale('log')
plt.title('Entropy rate')

In [ ]:
# for i in range(len(lamdas)):
#     plt.subplot(1, 2, 1)
#     plt.plot(static_net_heat.S[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)) + '  #Communities= ' + str(len(list_forw_flows[i].partition[0].cluster_list)))

#     plt.subplot(1, 2, 2)
#     plt.plot(static_net_heat.S[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)) + '  #Communities= ' + str(len(list_forw_flows[i].partition[0].cluster_list)), )
#     plt.yscale('log')
# plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
# plt.title('Entropy rate')
# #plt.savefig('./figures/fig3_recap_entropy_plots/fig3_entropy_rate.eps', format='eps')

### Spectral vNS

In [ ]:
for i in lamdas:
    static_net_heat.compute_spectral_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_heat.times[-1:])

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(lamdas, one_time(static_net_heat.spectral_vNS))

plt.subplot(1, 2, 2)
plt.plot(lamdas, one_time(static_net_heat.spectral_vNS))
plt.yscale('log')
plt.title('Spectral vNS')

In [ ]:
# for i in range(len(lamdas)):
#     plt.subplot(1, 2, 1)
#     plt.plot(static_net_heat.spectral_vNS[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)) + '  #Communities= ' + str(len(list_forw_flows[i].partition[0].cluster_list)))

#     plt.subplot(1, 2, 2)
#     plt.plot(static_net_heat.spectral_vNS[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)) + '  #Communities= ' + str(len(list_forw_flows[i].partition[0].cluster_list)), )
#     plt.yscale('log')
# plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
# plt.title('Spectral vNS')
# #plt.savefig('./figures/fig3_recap_entropy_plots/fig3_spectral_vNS.eps', format='eps')

### New vNS

In [ ]:
# for i in lamdas:
#     static_net_heat.compute_new_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_heat.times[-1:])

In [ ]:
# plt.subplot(1, 2, 1)
# plt.plot(lamdas, one_time(static_net_heat.new_vNS))

# plt.subplot(1, 2, 2)
# plt.plot(lamdas, one_time(static_net_heat.new_vNS))
# plt.yscale('log')
# plt.title('New vNS')

In [ ]:
# for i in range(len(lamdas)):
#     plt.subplot(1, 2, 1)
#     plt.plot(static_net_heat.new_vNS[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)) + '  #Communities= ' + str(len(list_forw_flows[i].partition[0].cluster_list)))

#     plt.subplot(1, 2, 2)
#     plt.plot(static_net_heat.new_vNS[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)) + '  #Communities= ' + str(len(list_forw_flows[i].partition[0].cluster_list)), )
#     plt.yscale('log')
# plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
# plt.title('New vNS')
# #plt.savefig('./figures/fig3_recap_entropy_plots/fig3_new_vNS.eps', format='eps')

### Simple vNS

In [ ]:
# for i in lamdas:
#     static_net_heat.compute_simple_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_heat.times[-1:])

In [ ]:
# plt.subplot(1, 2, 1)
# plt.plot(lamdas, one_time(static_net_heat.simple_vNS))

# plt.subplot(1, 2, 2)
# plt.plot(lamdas, one_time(static_net_heat.simple_vNS))
# plt.yscale('log')
# plt.title('Simple vNS')

In [ ]:
for i in range(len(lamdas)):
    plt.subplot(1, 2, 1)
    plt.plot(static_net_heat.simple_vNS[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)) + '  #Communities= ' + str(len(list_forw_flows[i].partition[0].cluster_list)))

    plt.subplot(1, 2, 2)
    plt.plot(static_net_heat.simple_vNS[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)) + '  #Communities= ' + str(len(list_forw_flows[i].partition[0].cluster_list)), )
    plt.yscale('log')
plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Simple vNS')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_simple_vNS.eps', format='eps')

### Von Neumann Entropy

In [ ]:
for i in lamdas:
    static_net_heat.compute_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_heat.times[-1:])

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(lamdas, one_time(static_net_heat.vNS))

plt.subplot(1, 2, 2)
plt.plot(lamdas, one_time(static_net_heat.vNS))
plt.yscale('log')
plt.title('vNS')

In [ ]:
for i in range(len(lamdas)):
    print(lamdas[i])
    plt.subplot(1, 2, 1)
    plt.plot(static_net_heat.vNS[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)))

    plt.subplot(1, 2, 2)
    plt.plot(static_net_heat.vNS[lamdas[i]], label='lamda = ' + str(round(lamdas[i], 3)))
    plt.yscale('log')
plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('vNS')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_vNS.eps', format='eps')

# Derivatives

In [ ]:
def derivative(v1, v2):
    assert len(v1) == len(v2), "Error: in order to compute the numerical derivative, the two varibales need to have the same lenght."
    derivative = []
    for i, vi in enumerate(v1[1:]):
        m1 =(vi - v1[i-1])/(v2[i]-v2[i-1])
        m2 =(v1[i+1] - vi)/(v2[i+1]-v2[i])
        derivative.append((m1+m2)/2)
    return derivative

In [ ]:
between_lamdas=np.linspace(-4,2,20)

In [ ]:
net.times

In [ ]:
sample_times = np.arange(0, len(net.times), 500)
print(sample_times)

### Entropy Rate

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_heat.S[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(net.times[t], 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(net.times[t], 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left',  bbox_to_anchor=(1.05, 1))
plt.title('d(Entropy rate) / dlamda')
plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dS_dlamda.eps', format='eps')

plt.show()

### Spectral vNS

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_heat.spectral_vNS[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(Spectral vNS) / dlamda')
plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dspectral_vNS_dlamda.eps', format='eps')

plt.show()

### Derivative new vNS

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_heat.new_vNS[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(New vNS) / dlamda')
plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dnew_vNS_dlamda.eps', format='eps')

plt.show()

### Simple vNS

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_heat.simple_vNS[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('d(Simple vNS) / dlamda')
plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dsimple_vNS_dlamda.eps', format='eps')

plt.show()

### Deriative vNS

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_heat.vNS[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(vNS) / dlamda')
plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dvNS_dlamda.eps', format='eps')

plt.show()

# Weighted Static Network

In [12]:
static_net_weighted_heat.times

Index([  0.085593365346635,  0.2568114494294681, 0.29259991312776745,
       0.33642992110991066,  0.4032220572650702,  0.5383267898761216,
        0.5506873897987057,  0.5816441701637541,  0.6002483938865569,
        0.9995373483327974,
       ...
         98.58265894152568,   98.82776357311806,   98.86598743445596,
         99.15813002774136,   99.27120716531084,   99.62534157567295,
         99.67338096814258,   99.75614328258465,   99.78440114428888,
         99.84367840864601],
      dtype='float64', name='times', length=998)

### Entropy Rate

In [ ]:
for i in lamdas:
    static_net_weighted_heat.compute_entropy(lamda=i, force_csr=True)

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_weighted_heat.S[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(net.times[t], 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(net.times[t], 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left',  bbox_to_anchor=(1.05, 1))
plt.title('d(Entropy rate) / dlamda')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dS_dlamda.eps', format='eps')

plt.show()

### Spectral vNS

In [ ]:
for i in lamdas:
    static_net_weighted_heat.compute_spectral_vonNeumann_entropy(lamda=i, force_csr=True)

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_weighted_heat.spectral_vNS[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(Spectral vNS) / dlamda')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dspectral_vNS_dlamda.eps', format='eps')

plt.show()

### New vNS

In [ ]:
for i in lamdas:
    static_net_weighted_heat.compute_new_vonNeumann_entropy(lamda=i, force_csr=True)

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_weighted_heat.new_vNS[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(New vNS) / dlamda')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dnew_vNS_dlamda.eps', format='eps')

plt.show()

### Simple vNS

In [ ]:
for i in lamdas:
    static_net_weighted_heat.compute_simple_vonNeumann_entropy(lamda=i, force_csr=True)

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_weighted_heat.simple_vNS[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('d(Simple vNS) / dlamda')
# plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dsimple_vNS_dlamda.eps', format='eps')

plt.show()

### vNS

In [ ]:
for i in lamdas:
    static_net_weighted_heat.compute_vonNeumann_entropy(lamda=i, force_csr=True)

In [ ]:
for i, t in enumerate(sample_times):

    St = []
    for j, lamda in enumerate(lamdas):
        St.append(static_net_weighted_heat.vNS[lamda][i])
    
    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

    plt.subplot(1, 2, 1)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))

    plt.subplot(1, 2, 2)
    plt.plot(lamdas[1:], ds_dt, label='t = ' + str(round(t, 3)))
    plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(vNS) / dlamda')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dvNS_dlamda.eps', format='eps')

plt.show()